In [0]:
#first, upload the file to the databricks file system (DBFS) in a Volume. This is manual

# converting a geospatial file to a topojson.

## Why convert to a TopoJSON? ##
Most geojson files are coordinate based. A topoJSON uses a series of arcs to store its geometry. Why use TopoJSON? Because PowerBI's Shape Map visulaization component only works with TopoJSON files.

Note you could do this processing on your desktop computer as well, it might help you to save it locally.

First, upload the file to the databricks file system (DBFS). This is manual.
Second, start a cluster. We have clusters with geospatial tools installed and that would that would help you out. But serverless is faster and cheaper and you can use pip install to do this.

In [0]:
%pip install geopandas
%pip install topojson
%pip install geojson

restart the kernal after installing libraries

In [0]:
dbutils.library.restartPython()

In [0]:
import geopandas as gpd
import pandas as pd
import topojson as tp
import geojson

In [0]:
filepath = "/Volumes/scorecard_fulcrum/scorecard_fulcrum_records/scorecard_fulcrum_volume/2020NTAMap.json"
filepath = "/Volumes/scorecard_fulcrum/geo/scoreacard_geospatial_files/community_districts_simplified.geojson"


#you can even do shapefiles.
filepath = "/Volumes/scorecard_fulcrum/geo/scoreacard_geospatial_files/BusinessImprovementDistrict.zip"
filepath = '/Volumes/moo_ops_workspace/geospatial/geospatial_files_volume/nypd.geojson'
filepath = "/Volumes/scorecard_fulcrum/geo/scoreacard_geospatial_files/PavementEdge_DSNY_Districts_Sections.zip"

In [0]:
#read the file. It can be a geojson, json, shapefile, any geopandas-compatible map.
gdf = gpd.read_file(filepath)
gdf.info()
#print(gdf['precinct'])

#you may need to simplify it if it is too large and you get a "not serializable error". The 0.0001 is the tolarance in degrees.
#0.0001 degrees is about 34 feet latitude, 27.8 feet longitude
gdf['geometry'] = gdf.simplify(0.0001)
#gdf.info()
#convert GeoDataFrame to GeoJSON
print("gdf simplified")
my_geojson = gdf.to_json()
print("gdf jsonified")



In [0]:
#convert GeoJSON to  (This takes a while)
my_topojson = tp.Topology(my_geojson)
print(my_topojson)
print("successfully converted to topojson")

# Write the file to the volume #
From there you may download it to your local computer.

In [0]:
filename = filepath.split("/")[-1].split('.')[0]
directory = filepath.split("/")[:-1]
directory = "/".join(directory)
print(directory)
topojson_path = f"{directory}/{filename}_topojson.json"
with open(topojson_path, 'w') as f:
    f.write(my_topojson.to_json())

regular_json_path = f"{directory}/{filename}_regular.json"
with open(regular_json_path, 'w') as f:
    f.write(my_geojson)

In [0]:
my_topojson

# Write the TopoJSON as a Delta Table (doesn't work)

In [0]:
my_geojson.show()

In [0]:
df = spark.read.json(topojson_path)
df.show()
destination_path = f"/scorecard_fulcrum/geo/{filename}"
destination_table = f"scorecard.geo.{filename}"
print(destination_path)
df.write.mode("overwrite").format("delta").saveAsTable(destination_table)

In [0]:
df = spark.read.json(regular_json_path)
destination_path = f"/scorecard_fulcrum/geo/{filename}_regular_json"
print(destination_path)
destination_table = f"scorecard.geo.{filename}_regular_json"
df.write.mode("overwrite").format("delta").saveAsTable(destination_table)

#lets try a different way
- convert geodataframe to a pandas dataframe
- convert pandas df to a spark df
- save spark df as delta table

## to be read by carto, geometry must be in well-known binary format ##

In [0]:
gdf.head()

In [0]:
import pandas as pd
from shapely import wkb
gdf2 = gdf.copy()
#ensure geometry column contains valid geometry objects
gdf2['geometry'] = gdf2['geometry'].apply(lambda geom: geom if geom.is_valid else geom.buffer(0))

#convert geometry column to WKB format
gdf2['geometry'] = gdf2['geometry'].apply(lambda geom: wkb.dumps(geom))

#convert GeoDataFrame to Pandas DataFrame
pdf = pd.DataFrame(gdf2)

#Convert Pandas Dataframe To Spark DataFrame
sdf = spark.createDataFrame(pdf)
sdf.write.mode("overwrite").format("delta").saveAsTable(f"scorecard_fulcrum.geo.{filename}_from_pandas_to_delta_wkb")

In [0]:
sdf.show()

# hmm this geometry doesnt' appear valid in Carto